In [14]:
#r "nuget: Jinaga"
#r "nuget: Jinaga.Store.SQLite"
#r "nuget: Jinaga.Notebooks"
#r "nuget: Jinaga.UnitTest"

using Jinaga;
using Jinaga.Extensions;
using Jinaga.Notebooks;
using Jinaga.Store.SQLite;
using Jinaga.UnitTest;
using System.Collections.Immutable;

Installed Packages Jinaga, 1.1.33 Jinaga.Notebooks, 1.1.8 Jinaga.Store.SQLite, 1.1.10 Jinaga.UnitTest, 1.1.4

# Jinaga Quick Start

Jinaga is a data management framework for mobile, web, and distributed applications.
It is particularly good for offline-first applications and progressive web apps, where data is stored locally and synchronized with a server.
We call that server a *replicator*, but we'll get to that.

## Jinaga Client

A Jinaga client manages data in your application and connects to a replicator.
Create a client using a factory method.
For example, to store local state in SQLite, use this method:

In [15]:
JinagaClient j = JinagaSQLiteClient.Create(options =>
{
    // Connect to the Jinaga replicator that you host yourself, or that we host for you.
    options.HttpEndpoint = new Uri("https://rep.jinaga.com/myreplicator");

    // Use the SQLite store to persist data locally.
    options.SQLitePath = System.IO.Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.LocalApplicationData),
        "jinaga", "myapplication.sqlite"
    );
});

Or to use Jinaga in a unit test, use an in-memory store:

In [16]:
JinagaClient j = JinagaTest.Create(options =>
{
    options.User = new User("--- TEST USER ---");
});

## Facts

The core unit of data in Jinaga is a *fact*.
A fact is an immutable record of an event, decision, or state change.
Let's start simple.
When the user logs into your application, that's a fact.

In [17]:
(User user, UserProfile profile) = await j.Login();

j.RenderFacts(user)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER ---

Each fact has a type (`Jinaga.User` in this case) and a set of properties (`publicKey` for this example).

The `Jinaga.User` type is defined in the Jinaga library.
But, of course, you can define your own types.
Write a record and use the `FactType` attribute to define a type.

In [18]:
[FactType("Construction.Project")]
public record Project(User creator, Guid id);

To create an instance of this fact, call `j.Fact`.
This method is asynchronous because it will save the fact to the local store and notify the replicator to synchronize it with other clients.

In [19]:
Project project = await j.Fact(new Project(user, Guid.NewGuid()));

j.RenderFacts(project)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- sdNFH8Eyjkq4ISbeoVLadd9xyu/dNI+OqxdSi20yWncrbYkTCt8nxZ2qdt1T87sHTwPVKhqHeV9JWhxu10oTCQ== -->
 
 sdNFH8Eyjkq4ISbeoVLadd9xyu/dNI+OqxdSi20yWncrbYkTCt8nxZ2qdt1T87sHTwPVKhqHeV9JWhxu10oTCQ== 
 
 Construction.Project 
 
 id 
 
 90f60be9-47a0-4efe-b... 
 
 
<!-- sdNFH8Eyjkq4ISbeoVLadd9xyu/dNI+OqxdSi20yWncrbYkTCt8nxZ2qdt1T87sHTwPVKhqHeV9JWhxu10oTCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 sdNFH8Eyjkq4ISbeoVLadd9xyu/dNI+OqxdSi20yWncrbYkTCt8nxZ2qdt1T87sHTwPVKhqHeV9JWhxu10oTCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

## Predecessors

When we defined the `Construction.Project` type, we used `Jinaga.User` as one of its properties.
That made it a *predecessor*.
In this case, the meaning of that predecessor relationship is that the user created the project.

Once you have predecessors, you can query for facts that are related to them.
These, as you might have guessed, are called *successors*.

Let's query for all projects that a user has created.

In [22]:
// Create a couple more projects.
await j.Fact(new Project(user, Guid.NewGuid()));
await j.Fact(new Project(user, Guid.NewGuid()));

var projectsCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
);

ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- sdNFH8Eyjkq4ISbeoVLadd9xyu/dNI+OqxdSi20yWncrbYkTCt8nxZ2qdt1T87sHTwPVKhqHeV9JWhxu10oTCQ== -->
 
 sdNFH8Eyjkq4ISbeoVLadd9xyu/dNI+OqxdSi20yWncrbYkTCt8nxZ2qdt1T87sHTwPVKhqHeV9JWhxu10oTCQ== 
 
 Construction.Project 
 
 id 
 
 90f60be9-47a0-4efe-b... 
 
 
<!-- sdNFH8Eyjkq4ISbeoVLadd9xyu/dNI+OqxdSi20yWncrbYkTCt8nxZ2qdt1T87sHTwPVKhqHeV9JWhxu10oTCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 sdNFH8Eyjkq4ISbeoVLadd9xyu/dNI+OqxdSi20yWncrbYkTCt8nxZ2qdt1T87sHTwPVKhqHeV9JWhxu10oTCQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- m9K8WT5GNc+g53RwSwP0po8INW4CkVXAQr9VxfnthfQqiVOL6PF0YZKOU/aQf0VPI4Z/cW/SMoPFHOOhNw3CVA== -->
 
 m9K8WT5GNc+g53RwSwP0po8INW4CkVXAQr9VxfnthfQqiVOL6PF0YZKOU/aQf0VPI4Z/cW/SMoPFHOOhNw3CVA== 
 
 Construction.Project 
 
 id 
 
 f05f3e67-dd5e-46d8-9... 
 
 
<!-- m9K8WT5GNc+g53RwSwP0po8INW4CkVXAQr9VxfnthfQqiVOL6PF0YZKOU/aQf0VPI4Z/cW/SMoPFHOOhNw3CVA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 m9K8WT5GNc+g53RwSwP0po8INW4CkVXAQr9VxfnthfQqiVOL6PF0YZKOU/aQf0VPI4Z/cW/SMoPFHOOhNw3CVA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- 9TRZhm0zHGIAMQGYhh8TfDzrkX697pd+TnA+/TPPVos/QvhApVu7lMLIXqEgWi8G7ZU2Gf3pHIBIcJLxcZO5Hw== -->
 
 9TRZhm0zHGIAMQGYhh8TfDzrkX697pd+TnA+/TPPVos/QvhApVu7lMLIXqEgWi8G7ZU2Gf3pHIBIcJLxcZO5Hw== 
 
 Construction.Project 
 
 id 
 
 8705dd45-7614-448c-8... 
 
 
<!-- 9TRZhm0zHGIAMQGYhh8TfDzrkX697pd+TnA+/TPPVos/QvhApVu7lMLIXqEgWi8G7ZU2Gf3pHIBIcJLxcZO5Hw==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 9TRZhm0zHGIAMQGYhh8TfDzrkX697pd+TnA+/TPPVos/QvhApVu7lMLIXqEgWi8G7ZU2Gf3pHIBIcJLxcZO5Hw==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator